In [108]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import spacy 
import optuna
import xgboost as xgb
import re
from spacy_langdetect import LanguageDetector
from spacy.language import Language

import sys; sys.path.insert(0, '../') #adds all the code we've written in src
from tqdm.notebook import trange, tqdm
from preprocessing import *

from sklearnex import patch_sklearn
patch_sklearn()

from sklearnex.svm import SVC
from sklearn.metrics import accuracy_score as accuracy
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score as accuracy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer as tfidf
from sklearn.preprocessing import MaxAbsScaler

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [109]:
@Language.factory('language_detector')
def language_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm", disable = ["tagger", "parser", "attribute_ruler", "ner"])
nlp.add_pipe('sentencizer')
nlp.add_pipe('language_detector', last=True) #2
spacy_tokenizer = nlp.tokenizer
docs = nlp.pipe(tweets)

In [230]:
df = pd.read_csv("../../cyberbullying_tweets/tweets_with_lang.csv", index_col = 0)
df_en = df[df.lang == 'en'].copy().reset_index(drop = True)
cat_dict = {}
for i, type in enumerate(df.cyberbullying_type.unique()):
    cat_dict[type] = i 
    
df_en["y"] = df_en.cyberbullying_type.map(cat_dict)
df_en

,tweet_text,cyberbullying_type,lang,score,y
0,"In other words #katandandre, your food was cra...",not_cyberbullying,en,0.999999,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,en,0.999994,0
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,en,0.999997,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,en,0.999996,0
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,en,0.999998,0
...,...,...,...,...,...
44615,"Black ppl aren't expected to do anything, depe...",ethnicity,en,0.999998,5
44616,Turner did not withhold his disappointment. Tu...,ethnicity,en,0.999995,5
44617,I swear to God. This dumb nigger bitch. I have...,ethnicity,en,0.999995,5
44618,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,en,0.857141,5


In [222]:
def test_preprocess(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    vectorizer = CountVectorizer(tokenizer=word_tokenize, analyzer = 'word', stop_words=None, ngram_range=(1,1), lowercase=False)
    vectorizer.fit(X_train)

    X_train_bow = vectorizer.transform(X_train)
    X_test_bow = vectorizer.transform(X_test)
    print("Input Dimensionality")
    print(X_train_bow.shape)
    print(X_test_bow.shape)
    print()

    log_reg = LR(max_iter=10000, solver = 'saga', 
                 multi_class='multinomial').fit(X_train_bow, y_train)
    y_pred = log_reg.predict(X_test_bow)
    print("BOW Embedding Classification Report")
    print(classification_report(y_test, y_pred))
    print(accuracy(y_test, y_pred ))
    print()

    tf_vectorizer = tfidf()
    tf_vectorizer.fit(X_train_bow)

    X_train_tfidf = tf_vectorizer.transform(X_train_bow)
    X_test_tfidf = tf_vectorizer.transform(X_test_bow)

    log_reg_tfidf = LR(max_iter=10000, solver = 'saga', 
                      multi_class='multinomial').fit(X_train_tfidf, y_train)
    y_pred_tfidf = log_reg_tfidf.predict(X_test_tfidf)
    
    print("TF-IDF Embedding Classification Report")
    print(classification_report(y_test, y_pred_tfidf))
    print(accuracy(y_test, y_pred_tfidf))

    
# test_preprocess(df_en.tweet_text, df_en.y)

In [231]:
#removing @'s
def remove_punctuation(text): 
    my_punct = ['!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '.',
           '/', ':', ';', '<', '=', '>', '?', '[', '\\', ']', '^', '_', 
           '`', '{', '|', '}', '~', '»', '«', '“', '”', '#']
    punct_pattern = re.compile("[" + re.escape("".join(my_punct)) + "]")
    return re.sub(punct_pattern, "", text) # the "-" symbol should remain)

def remove_nums(text_arr): 
    numbers = r"[0-9]+"
    return text_arr.str.replace(numbers, "")

def remove_links(text_arr): 
    url_regex = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
    return text_arr.str.replace(url_regex, "@LINK@")

def replace_ats_with_token(text_arr): 
    at_regex = r"@[^\s]+"
    return text_arr.str.replace(at_regex, "@")

tweets = df_en.tweet_text.copy()
tweets = replace_ats_with_token(tweets)
tweets = remove_links(tweets)
tweets = to_lowercase(tweets)
#tweets = expand_contractions(tweets)
tweets = tweets.apply(remove_punctuation)
# tweets = remove_emoji(tweets)
# tweets = remove_nums(tweets)
# tweets = stem_series(tweets)
# tweets = lemmatize_series(tweets)



C:\Users\Ayush\AppData\Local\Temp/ipykernel_65712/477408584.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  return text_arr.str.replace(at_regex, "@")
C:\Users\Ayush\AppData\Local\Temp/ipykernel_65712/477408584.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  return text_arr.str.replace(url_regex, "@LINK@")


In [157]:
test_preprocess(tweets, df_en.y)

C:\Users\Ayush\.conda\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Input Dimensionality
(35696, 42873)
(8924, 42873)

BOW Embedding Classification Report
              precision    recall  f1-score   support

           0       0.59      0.52      0.56      1399
           1       0.90      0.84      0.87      1517
           2       0.96      0.94      0.95      1587
           3       0.58      0.70      0.63      1364
           4       0.97      0.97      0.97      1544
           5       0.99      0.98      0.98      1513

    accuracy                           0.83      8924
   macro avg       0.83      0.83      0.83      8924
weighted avg       0.84      0.83      0.84      8924

0.8342671447781264

TF-IDF Embedding Classification Report
              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1399
           1       0.90      0.83      0.86      1517
           2       0.93      0.95      0.94      1587
           3       0.58      0.71      0.64      1364
           4       0.94      0.98      0

In [232]:
df_en["processed"] = tweets

In [233]:
df_en[df_en.processed.duplicated()].to_csv("../../cyberbullying_tweets/duplicated.csv")

In [234]:
df_unduped = df_en.drop_duplicates("processed").copy()
df_unduped.groupby("cyberbullying_type").count()

,tweet_text,lang,score,y,processed
cyberbullying_type,,,,,
age,7901,7901,7901,7901,7901
ethnicity,7323,7323,7323,7323,7323
gender,7420,7420,7420,7420,7420
not_cyberbullying,6587,6587,6587,6587,6587
other_cyberbullying,5580,5580,5580,5580,5580
religion,7948,7948,7948,7948,7948


In [235]:
test_preprocess(df_unduped.processed, df_unduped.y)

C:\Users\Ayush\.conda\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Input Dimensionality
(34207, 43072)
(8552, 43072)

BOW Embedding Classification Report
              precision    recall  f1-score   support

           0       0.62      0.65      0.64      1362
           1       0.90      0.85      0.87      1495
           2       0.96      0.94      0.95      1525
           3       0.60      0.63      0.62      1121
           4       0.97      0.98      0.97      1566
           5       0.98      0.97      0.98      1483

    accuracy                           0.85      8552
   macro avg       0.84      0.84      0.84      8552
weighted avg       0.85      0.85      0.85      8552

0.8500935453695042

TF-IDF Embedding Classification Report
              precision    recall  f1-score   support

           0       0.62      0.63      0.62      1362
           1       0.90      0.84      0.87      1495
           2       0.94      0.95      0.94      1525
           3       0.59      0.62      0.61      1121
           4       0.96      0.97      0

In [236]:
unprocessed_tweets = df_en.tweet_text.copy()
test_preprocess(df_unduped.tweet_text, df_unduped.y)

C:\Users\Ayush\.conda\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Input Dimensionality
(34207, 65425)
(8552, 65425)

BOW Embedding Classification Report
              precision    recall  f1-score   support

           0       0.64      0.66      0.65      1362
           1       0.91      0.86      0.88      1495
           2       0.95      0.95      0.95      1525
           3       0.63      0.67      0.65      1121
           4       0.98      0.97      0.97      1566
           5       0.98      0.97      0.98      1483

    accuracy                           0.86      8552
   macro avg       0.85      0.85      0.85      8552
weighted avg       0.86      0.86      0.86      8552

0.8574602432179607

TF-IDF Embedding Classification Report
              precision    recall  f1-score   support

           0       0.66      0.63      0.64      1362
           1       0.89      0.87      0.88      1495
           2       0.92      0.96      0.94      1525
           3       0.62      0.65      0.64      1121
           4       0.96      0.97      0

In [216]:
def get_number_of_ats(tweet): 
    at_regex = r"@"
    ats = re.findall(at_regex, tweet)
    return len(ats)

def get_number_of_hashtags(tweet): 
    return len(re.findall(r"#", tweet))

def is_retweet(tweet): 
    return len(re.findall(r"rt", tweet))

df_en["num_ats"] = df_en.tweet_text.apply(lambda row: get_number_of_ats(row))
df_en["num_tags"] = df_en.tweet_text.apply(lambda row: get_number_of_hashtags(row))
df_en["rt"] = df_en.tweet_text.apply(lambda row: is_retweet(row))

cols = ["cyberbullying_type", "num_ats", "score", "num_tags", "rt"]
df_en[cols].groupby("cyberbullying_type").mean()

,num_ats,score,num_tags,rt
cyberbullying_type,,,,
age,0.065206,0.997388,0.093278,0.197508
ethnicity,0.611028,0.977364,0.141444,0.131960
gender,0.570849,0.988996,0.342405,0.179249
not_cyberbullying,0.780499,0.978211,0.410184,0.121099
other_cyberbullying,0.860868,0.982929,0.206333,0.122510
religion,0.519433,0.997447,0.225677,0.379012


In [215]:
lengths = df_en.processed.apply(lambda x: len(x)).apply(lambda x: x <= 20).sum()
unduped_lengths = df_unduped.processed.apply(lambda x: len(x)).apply(lambda x: x <= 20).sum()

print(lengths)
print(unduped_lengths)

1785
1314


In [205]:
X_train, X_test, y_train, y_test = train_test_split(df_unduped.tweet_text, df_unduped.y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.175, random_state=7 )

vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,1))
vectorizer.fit(X_train)

tf_vectorizer = tfidf()

X_train_bow = vectorizer.transform(X_train)
X_val_bow = vectorizer.transform(X_val)
X_test_bow = vectorizer.transform(X_test)

tf_vectorizer.fit(X_train_bow)

X_train_tfidf = tf_vectorizer.transform(X_train_bow)
X_val_tfidf = tf_vectorizer.transform(X_val_bow)
X_test_tfidf = tf_vectorizer.transform(X_test_bow)

scaler = MaxAbsScaler()
scaler.fit(X_train_tfidf)

X_train_tfidf = scaler.transform(X_train_tfidf)
X_val_tfidf = scaler.transform(X_val_tfidf)
X_test_tfidf = scaler.transform(X_test_tfidf)

In [144]:
vectorizer.get_feature_names_out()
top = np.argmax(X_train_bow.sum(axis = 0))
vectorizer.get_feature_names_out().shape

ind = np.argpartition(X_train_bow.sum(axis = 0), -20000)[0,:-300]
for top in vectorizer.get_feature_names_out()[ind]:
    print(list(top))

['horseshit', 'inconsistency', 'timesyou', 'timesup', 'timesofisrael', 'timesiamthankfulforamask', 'timesbut', 'timesaver', 'timesas', 'timezone', 'timers', 'timenotedhtml', 'timely', 'timelinei', 'timeisup', 'timeframe', 'timebecause', 'timcad', 'timbersarmy', 'timbaland', 'timm', 'tikka', 'timmy', 'tii', 'tigrisxxx', 'tighty', 'tightened', 'tighmir', 'tigers', 'tiffydianne', 'tiffsmells', 'tiffalips', 'timonen', 'tienen', 'tiene', 'tiempo', 'tieflings', 'tidies', 'tidal', 'ticsterrorizinghumanitypolicemany', 'tickling', 'tickles', 'timpayton', 'timrigby', 'tichacekwill', 'tice', 'tiburke', 'tibo', 'tiara', 'tian', 'tiaaguwop', 'thâ', 'thz', 'thyself', 'timwisebookclub', 'thwy', 'thwarted', 'thw', 'thursdaymotivation', 'thursdaymorning', 'thunkit', 'thunderfury', 'thundercats', 'thunderball', 'thumbs', 'tinbarka', 'thuglife', 'thuggin', 'thuggerrr', 'thuggatee', 'thugdawg', 'tindakan', 'thucydides', 'ththth', 'ththinkr', 'thth', 'tinfoil', 'thst', 'thse', 'thry', 'thruthfulness', 'thr

In [94]:
vectorizer.get_feature_names_out()

array(['aa', 'aaa', 'aaaa', ..., 'ᵗʳᵃⁱˡ', 'ᵗʳⁱᵍᵍᵉʳ', 'ᶠⁱⁿᵍᵉʳ'],
      dtype=object)

In [12]:
#bow optimization
def objective(trial): 
    kernels = ["linear"]
    kernel = trial.suggest_categorical("kernel", kernels)
    regularization = trial.suggest_float("regularization", 0, 2)
    #gamma = trial.suggest_loguniform("gamma", 10e-3, 10e3)
    #degree = trial.suggest_int("degree",2,5)
    
    model = SVC(max_iter = 100, C = regularization, 
                kernel = kernel).fit(X_train_bow, y_train)
    
    y_pred = model.predict(X_val_bow)
    return accuracy(y_val, y_pred)

study = optuna.create_study(study_name = "SVC_BOW", direction = "maximize")
study.optimize(objective, n_trials = 35)
print(study.best_trial)

[I 2022-04-03 22:25:17,072] A new study created in memory with name: SVC_BOW
[I 2022-04-03 22:25:36,455] Trial 0 finished with value: 0.8288641156974993 and parameters: {'kernel': 'linear', 'regularization': 0.9551356605586845}. Best is trial 0 with value: 0.8288641156974993.
[I 2022-04-03 22:25:49,468] Trial 1 finished with value: 0.8348900271166014 and parameters: {'kernel': 'linear', 'regularization': 0.4181296627682145}. Best is trial 1 with value: 0.8348900271166014.
[I 2022-04-03 22:26:06,248] Trial 2 finished with value: 0.8255498644169931 and parameters: {'kernel': 'linear', 'regularization': 1.48401744818691}. Best is trial 1 with value: 0.8348900271166014.
[I 2022-04-03 22:26:21,875] Trial 3 finished with value: 0.82389273877674 and parameters: {'kernel': 'linear', 'regularization': 1.9854560812580309}. Best is trial 1 with value: 0.8348900271166014.
[I 2022-04-03 22:26:36,304] Trial 4 finished with value: 0.8263031033443808 and parameters: {'kernel': 'linear', 'regularizatio

FrozenTrial(number=33, values=[0.8478457366676709], datetime_start=datetime.datetime(2022, 4, 3, 22, 32, 0, 944431), datetime_complete=datetime.datetime(2022, 4, 3, 22, 32, 10, 622652), params={'kernel': 'linear', 'regularization': 0.1497481394334123}, distributions={'kernel': CategoricalDistribution(choices=('linear',)), 'regularization': UniformDistribution(high=2.0, low=0.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=33, state=TrialState.COMPLETE, value=None)


In [206]:
model = SVC(max_iter = 10000, C = 0.1497481394334123, 
                kernel = "linear").fit(X_train_bow, y_train)

y_pred = model.predict(X_test_bow)

print(classification_report(y_test, y_pred))
print(accuracy(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.67      0.65      1020
           1       0.93      0.86      0.89      1103
           2       0.97      0.95      0.96      1194
           3       0.63      0.65      0.64       827
           4       0.97      0.97      0.97      1198
           5       0.97      0.98      0.98      1070

    accuracy                           0.86      6412
   macro avg       0.85      0.85      0.85      6412
weighted avg       0.87      0.86      0.86      6412

0.8601060511540861


In [79]:
# tfidf optimization
def objective(trial): 
    kernels = ["linear", "poly", "rbf", "sigmoid"]
    kernel = trial.suggest_categorical("kernel", kernels)
    regularization = trial.suggest_float("regularization", 0, 1)
    degree = trial.suggest_int("degree", 1, 8)
    
    model = SVC(max_iter = 10000, C = regularization, 
                kernel = kernel, degree = degree).fit(X_train_tfidf, y_train)
    
    y_pred = model.predict(X_val_tfidf)
    return accuracy(y_val, y_pred)

study = optuna.create_study(study_name = "SVC_Accuracy", direction = "maximize")
study.optimize(objective, n_trials = 35)
print(study.best_trial)


[I 2022-04-03 21:25:50,551] A new study created in memory with name: SVC_Accuracy
[I 2022-04-03 21:26:30,598] Trial 0 finished with value: 0.201416089183489 and parameters: {'kernel': 'poly', 'regularization': 0.686954021342695, 'degree': 8}. Best is trial 0 with value: 0.201416089183489.
[I 2022-04-03 21:26:45,307] Trial 1 finished with value: 0.8424224163904791 and parameters: {'kernel': 'sigmoid', 'regularization': 0.6919440187969182, 'degree': 6}. Best is trial 1 with value: 0.8424224163904791.
[I 2022-04-03 21:27:41,515] Trial 2 finished with value: 0.22777945164206087 and parameters: {'kernel': 'poly', 'regularization': 0.8812559130767845, 'degree': 5}. Best is trial 1 with value: 0.8424224163904791.
[I 2022-04-03 21:28:25,099] Trial 3 finished with value: 0.7431455257607713 and parameters: {'kernel': 'poly', 'regularization': 0.8372090228566549, 'degree': 2}. Best is trial 1 with value: 0.8424224163904791.
[I 2022-04-03 21:28:43,577] Trial 4 finished with value: 0.84528472431455

[I 2022-04-03 21:39:00,062] Trial 36 finished with value: 0.8454353721000302 and parameters: {'kernel': 'sigmoid', 'regularization': 0.9418293817849873, 'degree': 5}. Best is trial 22 with value: 0.8455860198855076.
[I 2022-04-03 21:39:17,462] Trial 37 finished with value: 0.8445314853871648 and parameters: {'kernel': 'sigmoid', 'regularization': 0.8622384294294669, 'degree': 7}. Best is trial 22 with value: 0.8455860198855076.
[I 2022-04-03 21:39:57,249] Trial 38 finished with value: 0.2270262127146731 and parameters: {'kernel': 'poly', 'regularization': 0.799536764144397, 'degree': 5}. Best is trial 22 with value: 0.8455860198855076.
[I 2022-04-03 21:40:10,933] Trial 39 finished with value: 0.8449834287435974 and parameters: {'kernel': 'sigmoid', 'regularization': 0.9194315880908418, 'degree': 5}. Best is trial 22 with value: 0.8455860198855076.
[I 2022-04-03 21:40:35,347] Trial 40 finished with value: 0.8210304308526665 and parameters: {'kernel': 'rbf', 'regularization': 0.945370814

KeyboardInterrupt: 

In [207]:
model = SVC(max_iter = 10000, C = 0.9775672385207569, 
                kernel = "sigmoid").fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))
print(accuracy(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.61      0.62      1020
           1       0.92      0.86      0.89      1103
           2       0.96      0.96      0.96      1194
           3       0.61      0.66      0.64       827
           4       0.95      0.98      0.96      1198
           5       0.97      0.96      0.96      1070

    accuracy                           0.85      6412
   macro avg       0.84      0.84      0.84      6412
weighted avg       0.85      0.85      0.85      6412

0.8523081721771678


In [208]:
#5 class classification for fun 
df5 = df_unduped[df_unduped.y != 0].copy()

tweets = df5.tweet_text.copy()
tweets = tweets.apply(remove_punctuation)
tweets = to_lowercase(tweets)

X_train, X_test, y_train, y_test = train_test_split(tweets, df5.y, test_size=0.20, random_state=42)

vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,1))
vectorizer.fit(X_train)

X_train_bow = vectorizer.transform(X_train)
X_test_bow = vectorizer.transform(X_test)

model = SVC(max_iter = 10000, C = 0.1497481394334123, 
            kernel = "linear").fit(X_train_bow, y_train)

y_pred = model.predict(X_test_bow)


print(classification_report(y_test, y_pred))
print(accuracy(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.94      0.90      0.92      1513
           2       0.99      0.96      0.98      1597
           3       0.81      0.91      0.86      1091
           4       0.99      0.99      0.99      1556
           5       0.98      0.98      0.98      1475

    accuracy                           0.95      7232
   macro avg       0.94      0.95      0.94      7232
weighted avg       0.95      0.95      0.95      7232

0.9491150442477876


In [209]:
test_preprocess(tweets, df5.y)

C:\Users\Ayush\.conda\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Input Dimensionality
(28927, 48609)
(7232, 48609)

BOW Embedding Classification Report
              precision    recall  f1-score   support

           1       0.94      0.91      0.92      1513
           2       0.99      0.97      0.98      1597
           3       0.83      0.91      0.87      1091
           4       0.99      0.99      0.99      1556
           5       0.99      0.98      0.98      1475

    accuracy                           0.95      7232
   macro avg       0.95      0.95      0.95      7232
weighted avg       0.96      0.95      0.95      7232

0.9535398230088495

TF-IDF Embedding Classification Report
              precision    recall  f1-score   support

           1       0.94      0.90      0.92      1513
           2       0.97      0.98      0.97      1597
           3       0.84      0.86      0.85      1091
           4       0.99      0.99      0.99      1556
           5       0.97      0.98      0.98      1475

    accuracy                           

In [41]:
D_train = xgb.DMatrix(X_train_bow, label=y_train - 1)
D_test = xgb.DMatrix(X_test_bow, label=y_test - 1)

def objective(trial): 
    param = {
        'eta': trial.suggest_loguniform("eta", 10e-4, 1), 
        'max_depth': trial.suggest_int("max_depth", 2, 6),  
        'booster': trial.suggest_categorical('booster', ["gbtree", "dart"]),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'objective': 'multi:softprob',  
        'num_class': 5} 

    steps = 150

    model = xgb.train(param, D_train, steps)
    y_pred_prob = model.predict(D_test)
    y_pred = np.argmax(y_pred_prob, axis = 1)
    
    return accuracy(y_test - 1, y_pred)

study = optuna.create_study(study_name = "XGB_BOW", direction = "maximize")
study.optimize(objective, n_trials = 35)
print(study.best_trial)

[I 2022-04-03 23:27:48,977] A new study created in memory with name: XGB_BOW


[23:27:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-03 23:27:59,506] Trial 0 finished with value: 0.9487145682267634 and parameters: {'eta': 0.5293444530559384, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.005002242517786541, 'alpha': 0.0002638537621423466}. Best is trial 0 with value: 0.9487145682267634.


[23:27:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-03 23:30:22,020] Trial 1 finished with value: 0.9075807514831905 and parameters: {'eta': 0.02635201034638823, 'max_depth': 4, 'booster': 'dart', 'lambda': 7.733056107376065e-05, 'alpha': 1.896717902418352e-05}. Best is trial 0 with value: 0.9487145682267634.


...

[23:48:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-03 23:48:47,919] Trial 33 finished with value: 0.9499011206328279 and parameters: {'eta': 0.5206589966122016, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.007050744703206729, 'alpha': 2.9080327558089826e-05}. Best is trial 32 with value: 0.9501647989452867.


[23:48:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-03 23:51:15,678] Trial 34 finished with value: 0.9489782465392221 and parameters: {'eta': 0.5886267996338065, 'max_depth': 6, 'booster': 'dart', 'lambda': 0.03279628677276571, 'alpha': 2.6199016760329755e-05}. Best is trial 32 with value: 0.9501647989452867.


FrozenTrial(number=32, values=[0.9501647989452867], datetime_start=datetime.datetime(2022, 4, 3, 23, 48, 16, 217182), datetime_complete=datetime.datetime(2022, 4, 3, 23, 48, 32, 91063), params={'eta': 0.4211206745599294, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.014646287896012975, 'alpha': 2.4918523070189394e-05}, distributions={'eta': LogUniformDistribution(high=1.0, low=0.001), 'max_depth': IntUniformDistribution(high=6, low=2, step=1), 'booster': CategoricalDistribution(choices=('gbtree', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=32, state=TrialState.COMPLETE, value=None)


In [43]:
study.optimize(objective, n_trials=250)

[23:54:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-03 23:54:53,276] Trial 37 finished with value: 0.9355306526038233 and parameters: {'eta': 0.044190361739635295, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.01214564882506794, 'alpha': 2.4526527857957247e-05}. Best is trial 32 with value: 0.9501647989452867.


[23:54:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-03 23:56:43,643] Trial 38 finished with value: 0.9152274225444957 and parameters: {'eta': 0.016668376430795693, 'max_depth': 6, 'booster': 'dart', 'lambda': 0.001462032343714355, 'alpha': 0.02663552033995146}. Best is trial 32 with value: 0.9501647989452867.


[23:56:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[I 2022-04-04 01:28:09,950] Trial 284 finished with value: 0.9492419248516809 and parameters: {'eta': 0.40251735227600643, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.6525771285354327, 'alpha': 0.0012790900542425006}. Best is trial 219 with value: 0.9510876730388925.


[01:28:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-04 01:28:21,371] Trial 285 finished with value: 0.9497692814765986 and parameters: {'eta': 0.7939413478226438, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.23971681374652976, 'alpha': 0.02306902581782106}. Best is trial 219 with value: 0.9510876730388925.


[01:28:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-04-04 01:28:33,028] Trial 286 finished with value: 0.948582729070534 and parameters: {'eta': 0.28042402830640767, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.34399697527470785, 'alpha': 0.00038283460022870146}. Best is trial 219 with value: 0.9510876730388925.


In [45]:
print(study.best_trial)

FrozenTrial(number=219, values=[0.9510876730388925], datetime_start=datetime.datetime(2022, 4, 4, 1, 13, 4, 568692), datetime_complete=datetime.datetime(2022, 4, 4, 1, 13, 16, 18452), params={'eta': 0.45130011045733887, 'max_depth': 6, 'booster': 'gbtree', 'lambda': 0.6254188197006147, 'alpha': 0.00030775521585501626}, distributions={'eta': LogUniformDistribution(high=1.0, low=0.001), 'max_depth': IntUniformDistribution(high=6, low=2, step=1), 'booster': CategoricalDistribution(choices=('gbtree', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=219, state=TrialState.COMPLETE, value=None)


In [210]:
vectorizer.get_feature_names_out()
top = np.argmax(X_train_bow.sum(axis = 0))
vectorizer.get_feature_names_out().shape


(46575,)

In [211]:
ind = np.argpartition(X_train_bow.sum(axis = 0), -300)[0,-300:]
for top in vectorizer.get_feature_names_out()[ind]:
    print(list(top))

['abuse', 'play', 'aint', 'story', 'sick', 'gets', 'calls', 'seeing', 'fake', 'watching', 'straight', 'looking', 'came', 'follow', 'weird', 'fucks', 'started', 'speak', 'hair', 'yo', 'police', 'place', 'nice', 'post', 'beat', 'doesn', 'guess', 'cool', 'kat', 'president', 'media', 'male', 'saw', 'far', 'self', 'words', 'hey', 'house', 'change', 'females', 'times', 'true', 'fun', 'video', 'mom', 'wont', 'money', 'maybe', 'jews', 'fat', 'hard', 'damn', 'theres', 'using', 'nigga', 'sorry', 'child', 'children', 'care', 'fight', 'means', 'sex', 'family', 'pakistan', 'theyre', 'head', 'end', 'gays', 'tweet', 'class', 'ago', 'best', 'understand', 'lmao', 'point', 'ones', 'help', 'watch', 'fact', 'killed', 'american', 'group', 'america', 'long', 'reason', 'killing', 'act', 'okay', 'kind', 'race', 'great', 'problem', 'non', 'seen', 'ill', 'probably', 'grade', 've', 'ask', 'matter', 'rights', 'start', 'stand', 'big', 'ignorant', 'hindu', 'arent', 'dude', 'gonna', 'war', 'boys', 'boy', 'hindus', '

In [82]:
ind

matrix([[36104, 47537,     2, ..., 18000, 26005, 37517]], dtype=int64)